## Limpando e manipulando dados após Web Scraping

In [1]:
import pandas as pd

In [2]:
import re

In [3]:
file = "datasets/imoveis.json"

df = pd.read_json(file, orient = 'records')

df.head()

,preco,dormitorios,area,vagas,bairro
0,[R$ 1.055.000],[3],[128],[3],"[Imóvel 106391 - , Apartamento, \n ..."
1,[R$ 1.112.000],[3],[136.34],[2],"[Imóvel 130624 - , Apartamento, \n ..."
2,[R$ 1.620.000],[3],[204.87],[3],"[Imóvel 192740 - , Apartamento, \n ..."
3,[R$ 1.190.000],[3],[241],[2],"[Imóvel 196741 - , Casa, \n ..."
4,[R$ 3.053.011],[4],[271.3],[4],"[Imóvel 183630 - , Apartamento, \n ..."


In [4]:
print('Dimensões do Dataframe:\n')
print(f'Linhas: {df.shape[0]}')
print(f'Colunas: {df.shape[1]}')

Dimensões do Dataframe:

Linhas: 600
Colunas: 5


In [5]:
df.dtypes

preco          object
dormitorios    object
area           object
vagas          object
bairro         object
dtype: object

In [6]:
df['preco']

0      [R$ 1.055.000]
1      [R$ 1.112.000]
2      [R$ 1.620.000]
3      [R$ 1.190.000]
4      [R$ 3.053.011]
            ...      
595      [R$ 860.000]
596      [R$ 325.000]
597    [R$ 1.800.000]
598    [R$ 1.750.000]
599    [R$ 1.380.000]
Name: preco, Length: 600, dtype: object

In [7]:
df['preco'].sample(20)

103      [R$ 980.000]
326    [R$ 1.280.000]
246    [R$ 1.350.000]
436      [R$ 899.739]
462      [R$ 850.000]
435      [R$ 704.499]
558      [R$ 213.000]
398      [R$ 691.000]
12       [R$ 269.000]
582      [R$ 450.000]
393      [R$ 710.000]
176      [R$ 350.000]
534    [R$ 1.700.000]
169      [R$ 365.000]
521      [R$ 490.000]
38     [R$ 1.609.000]
504      [R$ 636.000]
201      [R$ 157.500]
220    [R$ 1.930.000]
117      [R$ 295.000]
Name: preco, dtype: object

In [8]:
df['preco'] = df['preco'].apply(lambda x : re.sub('\D', '', str(x))).astype(float)

In [9]:
df['preco']

0      1055000.0
1      1112000.0
2      1620000.0
3      1190000.0
4      3053011.0
         ...    
595     860000.0
596     325000.0
597    1800000.0
598    1750000.0
599    1380000.0
Name: preco, Length: 600, dtype: float64

In [10]:
df['preco'].sample(10)

85     1300000.0
425     899739.0
598    1750000.0
301     495000.0
564     424000.0
175     926000.0
514     860000.0
117     295000.0
258     229000.0
577     650000.0
Name: preco, dtype: float64

In [11]:
df.dtypes

preco          float64
dormitorios     object
area            object
vagas           object
bairro          object
dtype: object

In [12]:
df.isnull().sum()

preco           0
dormitorios     8
area            0
vagas          21
bairro          0
dtype: int64

In [13]:
df['dormitorios'].fillna("0", inplace = True)

In [14]:
df['dormitorios'] = df['dormitorios'].apply(lambda x : re.sub('\D', '', str(x))).astype(int)

In [15]:
df['dormitorios'].value_counts()

3    319
2    184
1     52
4     34
0      8
6      2
5      1
Name: dormitorios, dtype: int64

In [16]:
df['area'] = df['area'].apply(lambda x : re.sub('\D', '', str(x))).astype(float)

In [17]:
df['area'].sample(20)

538     7148.0
20       149.0
134     3559.0
205    15099.0
380     6162.0
57      5145.0
196     8988.0
507      190.0
227     5581.0
102     7393.0
459     6869.0
368    99616.0
328     5761.0
147       86.0
521     8021.0
215      855.0
130     5788.0
539       84.0
370      693.0
400     6771.0
Name: area, dtype: float64

In [18]:
df['vagas'].fillna("0", inplace = True)

In [19]:
df['vagas'] = df['vagas'].apply(lambda x : re.sub('\D', '', str(x))).astype(int)

In [20]:
df['vagas'].value_counts()

1    264
2    231
3     45
4     30
0     21
6      6
5      2
8      1
Name: vagas, dtype: int64

In [21]:
df['bairro'].sample(10)

301    [Imóvel 11974 - , Apartamento, \n             ...
318    [Imóvel 157602 - , Apartamento, \n            ...
493    [Imóvel 120157 - , Apartamento, \n            ...
459    [Imóvel 198059 - , Apartamento, \n            ...
86     [Imóvel 193727 - , Apartamento, \n            ...
102    [Imóvel 9868 - , Apartamento, \n              ...
514    [Imóvel 166972 - , Apartamento, \n            ...
404    [Imóvel 174212 - , Apartamento, \n            ...
331    [Imóvel 168934 - , Casa, \n                   ...
245    [Imóvel 194250 - , Apartamento, \n            ...
Name: bairro, dtype: object

In [22]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub('\d', '', str(x))).astype(str)

In [23]:
df['bairro'].sample(10)

347    ['Imóvel  - ', 'Apartamento', '\n             ...
11     ['Imóvel  - ', 'Apartamento', '\n             ...
122    ['Imóvel  - ', 'Apartamento', '\n             ...
19     ['Imóvel  - ', 'Apartamento', '\n             ...
369    ['Imóvel  - ', 'Apartamento', '\n             ...
242    ['Imóvel  - ', 'Cobertura', '\n               ...
456    ['Imóvel  - ', 'Apartamento', '\n             ...
583    ['Imóvel  - ', 'Apartamento', '\n             ...
39     ['Imóvel  - ', 'Apartamento', '\n             ...
471    ['Imóvel  - ', 'Apartamento', '\n             ...
Name: bairro, dtype: object

In [24]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub('Imóvel  -', '', x))

In [25]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub(r'\\n', '', x))

In [26]:
df['bairro'].sample(10)

302    [' ', 'Apartamento', '                        ...
368    [' ', 'Apartamento', '                        ...
486    [' ', 'Apartamento', '                        ...
50     [' ', 'Apartamento', '                        ...
254    [' ', 'Sala', '                        Auxilia...
395    [' ', 'Apartamento', '                        ...
374    [' ', 'Apartamento', '                        ...
42     [' ', 'Apartamento', '                        ...
47     [' ', 'Apartamento', '                        ...
461    [' ', 'Casa', '                        Agronom...
Name: bairro, dtype: object

In [27]:
df['tipo'] = df['bairro'].apply(lambda x : re.split(',', x)[1])

In [28]:
df['tipo'].sample(10)

82      'Apartamento'
203     'Apartamento'
217            'Casa'
138     'Apartamento'
68      'Apartamento'
52             'Casa'
511     'Apartamento'
200            'Casa'
67             'Casa'
444       'Cobertura'
Name: tipo, dtype: object

In [29]:
df['bairro'] = df['bairro'].apply(lambda x : re.split(',', x)[2])

In [30]:
df['bairro']

0       '                        Central Parque
1          '                        Menino Deus
2           '                        Rio Branco
3              '                        Sarandi
4           '                        Bela Vista
                         ...                   
595        '                        Auxiliadora
596           '                        Tristeza
597         '                        Bela Vista
598      '                        Jardim Europa
599     '                        Três Figueiras
Name: bairro, Length: 600, dtype: object

In [31]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub('\s', '', x))

In [32]:
df['bairro']

0      'CentralParque
1         'MeninoDeus
2          'RioBranco
3            'Sarandi
4          'BelaVista
            ...      
595      'Auxiliadora
596         'Tristeza
597        'BelaVista
598     'JardimEuropa
599    'TrêsFigueiras
Name: bairro, Length: 600, dtype: object

In [33]:
df['tipo'] = df['tipo'].apply(lambda x : re.sub("'", "", x))
df['bairro'] = df['bairro'].apply(lambda x : re.sub("'", '', x))

In [42]:
df['tipo'].head()

0     Apartamento
1     Apartamento
2     Apartamento
3            Casa
4     Apartamento
Name: tipo, dtype: object

In [41]:
df['bairro'].head()

0    CentralParque
1       MeninoDeus
2        RioBranco
3          Sarandi
4        BelaVista
Name: bairro, dtype: object

In [34]:
df.nunique()

preco          366
dormitorios      7
area           490
vagas            8
bairro          72
tipo             8
dtype: int64

In [35]:
df['tipo'].value_counts()

 Apartamento    481
 Casa            73
 Cobertura       27
 Loft            10
 Sala             6
 Terreno          1
 Loja             1
 Flat             1
Name: tipo, dtype: int64

In [36]:
df.describe()

,preco,dormitorios,area,vagas
count,6.000000e+02,600.000000,600.000000,600.000000
mean,8.636065e+05,2.550000,8829.695000,1.725000
std,9.296298e+05,0.815644,14787.265011,1.015428
min,1.178000e+05,0.000000,42.000000,0.000000
25%,3.800000e+05,2.000000,685.750000,1.000000
50%,5.870000e+05,3.000000,6530.000000,2.000000
75%,9.710585e+05,3.000000,10249.000000,2.000000
max,9.117117e+06,6.000000,229098.000000,8.000000


In [37]:
df.groupby(by = 'bairro').preco.mean().sort_values(ascending = False)[:15]

bairro
MontSerrat          2.213700e+06
BelaVista           2.204248e+06
BelémNovo           2.134000e+06
PedraRedonda        1.950000e+06
JardimSãoPedro      1.900000e+06
TrêsFigueiras       1.667200e+06
Auxiliadora         1.588450e+06
JardimEuropa        1.455259e+06
CentralParque       1.244618e+06
MoinhosdeVento      1.242944e+06
ChácaradasPedras    1.221700e+06
RioBranco           1.190297e+06
Petrópolis          1.170219e+06
Agronomia           9.677143e+05
MeninoDeus          9.590551e+05
Name: preco, dtype: float64

In [38]:
df.loc[df['bairro'] == "BelaVista"]

,preco,dormitorios,area,vagas,bairro,tipo
4,3053011.0,4,2713.0,4,BelaVista,Apartamento
36,1349000.0,3,140.0,2,BelaVista,Apartamento
88,1699000.0,3,14015.0,2,BelaVista,Apartamento
114,1990000.0,3,18121.0,4,BelaVista,Apartamento
120,779000.0,2,8337.0,2,BelaVista,Apartamento
135,1400000.0,2,9904.0,2,BelaVista,Apartamento
159,540000.0,1,649.0,2,BelaVista,Apartamento
166,3420000.0,4,22602.0,4,BelaVista,Apartamento
188,3896550.0,3,51954.0,4,BelaVista,Apartamento
191,1699000.0,3,25062.0,2,BelaVista,Apartamento


In [39]:
df.to_csv('datasets/new_imoveis.csv', index = False)